<a href="https://colab.research.google.com/github/joderthibaud/my_streamline_app/blob/main/WikipediaFAQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#   ! pip install -q kaggle
# from google.colab import files
# files.upload()

In [ ]:
!pip freeze | grep kaggle

In [ ]:
import kagglehub

    # Authenticate
kagglehub.login()

In [ ]:
kagglehub.competition_download('tensorflow2-question-answering')

In [ ]:
# # Unzip train and test set. This will take approximately 7 minutes
# ! unzip tensorflow2-question-answering.zip

In [ ]:
!pip install simpletransformers==4.28.0

In [ ]:
import json
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re

In [ ]:
def get_line_of_data(file):
    line = file.readline()
    line = json.loads(line)

    return line


def get_question_and_document(line):
    question = line['question_text']
    text = line['document_text'].split(' ')
    annotations = line['annotations'][0]

    return question, text, annotations


def get_long_candidate(i, annotations, candidate):
    # check if this candidate is the correct answer
    if i == annotations['long_answer']['candidate_index']:
        label = True
    else:
        label = False

    # get place where long answer starts and ends in the document text
    long_start = candidate['start_token']
    long_end = candidate['end_token']

    return label, long_start, long_end


def form_data_row(question, label, text, long_start, long_end):
    row = {
        'question': question,
        'long_answer': ' '.join(text[long_start:long_end]),
        'is_long_answer': label,
    }

    return row


def load_data(file_path, questions_start, questions_end):
    rows = []

    with open(file_path, encoding='latin-1') as file:

        for i in tqdm(range(questions_start, questions_end)):
            line = get_line_of_data(file)
            question, text, annotations = get_question_and_document(line)

            for i, candidate in enumerate(line['long_answer_candidates']):
                label, long_start, long_end = get_long_candidate(i, annotations, candidate)

                if label == True or (i % SAMPLE_RATE == 0):
                    rows.append(
                        form_data_row(question, label, text, long_start, long_end)
                    )

    return pd.DataFrame(rows)

In [ ]:
TRAIN_PATH = "/root/.cache/kagglehub/competitions/tensorflow2-question-answering/simplified-nq-train.jsonl"
TRAINING_SIZE, VALIDATION_SIZE = 1000, 1000
SAMPLE_RATE = 15

# Read train and validation sets
train_df = load_data(TRAIN_PATH, 0, TRAINING_SIZE)
val_df = load_data(TRAIN_PATH, TRAINING_SIZE, TRAINING_SIZE+VALIDATION_SIZE)

In [ ]:
!ls /root/.cache/kagglehub/competitions/tensorflow2-question-answering/

In [ ]:
def remove_html(sentence):
    html = re.compile(r'<.*?>')
    return html.sub(r'', sentence)


def clean_df(df):
    df['long_answer'] = df['long_answer'].apply(lambda x : remove_html(x))
    df['question'] = df['question'].apply(lambda x : remove_html(x))

    return df

In [ ]:
train_df = clean_df(train_df)
val_df = clean_df(val_df)

In [ ]:
display(train_df.head())
display(val_df.head())

In [16]:
from simpletransformers.classification import ClassificationModel

ImportError: cannot import name 'SequenceSummary' from 'transformers.modeling_utils' (/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py)